In [ ]:
# @title Fetch users of Tokyo from GIT API having more than 200 followers {"vertical-output":true}
import requests
import csv
import re, datetime

log_enable = False
range_max=7
per_page="100"
total_users =0
url = "https://api.github.com/search/users?q=location:Tokyo+followers:>200&per_page="+per_page
headers = {
    "Authorization": "Bearer GITHUB_TOKEN"
}

with open("users_raw.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["login", "name", "company","location","email","hireable","bio","public_repos","followers","following","created_at"])

    for page in range(1, range_max):
        paginated_url = f"{url}&page={page}"
        response = requests.get(paginated_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            users = data.get("items", [])
            total_users = total_users + users.__len__()
            print("Navigating Page ",page, " Users: ",  users.__len__())

            for user in users:

                user_details = requests.get(user["url"], headers=headers).json()
                ##print(user_details)
                fullname = user_details.get("name", "Not Available")
                company = user_details.get("company", "Not Available")
                location = user_details.get("location", "Not Available")
                email = user_details.get("email", "Not Available")
                hireable = user_details.get("hireable", "Not Available")
                bio = user_details.get("bio", "Not Available")
                public_repos = user_details.get("public_repos", "Not Available")
                followers = user_details.get("followers")
                following = user_details.get("following")
                created_on = user_details.get("created_at", "Not Available")
                extr_date = re.search('\d{4}-\d{2}-\d{2}', created_on)
                created_date = datetime.datetime.strptime(extr_date.group(), '%Y-%m-%d').date()

                if(log_enable):
                  print("Username ",user["login"])
                  print("Full name ", fullname)
                  print("Company ", company)
                  print("Location ", location)
                  print("Email ", email)
                  print("Bio ", bio)
                  print("Public Repos ", public_repos)
                  print("Followers ",followers)
                  print("Following ",following)
                  print("Created On", created_date)
                  print("-----------")
                else:
                  writer.writerow([user["login"],fullname, company, location,email, hireable,bio,public_repos, followers,following, created_on])

        else:
            print(f"Failed to retrieve data on page {page}: {response.status_code}")
            break
    print("Total Users ",total_users)

Navigating Page  1  Users:  100
Navigating Page  2  Users:  100
Navigating Page  3  Users:  100
Navigating Page  4  Users:  100
Navigating Page  5  Users:  100
Navigating Page  6  Users:  42
Total Users  542


In [ ]:
# @title Load the users.csv file as df {"vertical-output":true}
from os import replace
import pandas as pd
users_df = pd.read_csv("users_raw.csv")
print(users_df.head(5))
users_df =users_df.fillna('')
users_df['company'] = users_df['company'].str.upper().str.strip().str.replace('@','',regex=False)
print(users_df.head(5))
output_file_path = 'users.csv'
users_df.to_csv(output_file_path, index=False)
print(output_file_path, "Total Count ",users_df.shape)

        login              name             company      location  \
0  dennybritz       Denny Britz                 NaN  Tokyo, Japan   
1    wasabeef     Daichi Furiya    CyberAgent, Inc.  Tokyo, Japan   
2     dai-shi       Daishi Kato          Freelancer         Tokyo   
3      rui314        Rui Ueyama  Blue Whale Systems         Tokyo   
4     domenic  Domenic Denicola              Google  Tokyo, Japan   

              email hireable  \
0               NaN      NaN   
1               NaN     True   
2               NaN     True   
3  rui314@gmail.com      NaN   
4      d@domenic.me      NaN   

                                                 bio  public_repos  followers  \
0  High-school dropout. Ex Google Brain, Stanford...            61       9471   
1               Google Developers Expert for Android            46       9368   
2  React library author, maintaining three state ...           128       6945   
3                                                NaN            44  

In [2]:
users_df = pd.read_csv("users.csv")
users_df = users_df[users_df['location'].str.contains('Tokyo', na=False)]
top_5users = users_df.sort_values(by='created_at', ascending=True).head(5)['login'].tolist()
print (top_5users)

['kana', 'kakutani', 'mootoh', 'lhl', 'walf443']


In [ ]:
comp =users_df['company'].value_counts().sort_values(ascending=False)

#licence = licence.sort_values(by =login,asc='False')
comp

,count
company,
,206
GOOGLE,12
THE UNIVERSITY OF TOKYO,8
FREELANCE,7
"CYBERAGENT, INC.",5
...,...
"SYNCHROBO, INC",1
CYBERAGENT INC. AILAB,1
NEWMO,1


In [ ]:
# @title Fetch Repositories for each user from GITAPI
import csv
import requests
log_enable = False
# Define the GitHub API endpoint
BASE_URL = "https://api.github.com/users"

headers = {
    "Authorization": "Bearer GITHUB_TOKEN"
}

# Function to fetch repositories for a given user
def fetch_repositories(username):
    repos = []
    url = f"{BASE_URL}/{username}/repos"

    params = {
        "per_page": 100,
        "sort": "updated",
        "direction": "desc"
    }

    while url:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching repositories for {username}: {response.status_code}")
            break

        data = response.json()
        repos.extend(data)

        # Check for pagination
        if 'Link' in response.headers:
            links = response.headers['Link'].split(", ")
            url = None  # Reset URL for the next iteration
            for link in links:
                if 'rel="next"' in link:
                    url = link.split(";")[0].strip('<>')
                    break
        else:
            break

    return repos

# Function to extract relevant fields
def extract_repo_data(repos, username):
    extracted_data = []
    for repo in repos:
        extracted_data.append({
            "login": username,
            "full_name": repo.get("full_name"),
            "created_at": repo.get("created_at"),
            "stargazers_count": repo.get("stargazers_count"),
            "watchers_count": repo.get("watchers_count"),
            "language": repo.get("language"),
            "has_projects": repo.get("has_projects"),
            "has_wiki": repo.get("has_wiki"),
            "license_name": repo.get("license").get("name") if repo.get("license") else None
        })
    return extracted_data

# Main function to process the CSV
def main():
    all_repo_data = []
    user_fetch_count = 0
    total_repo_count = 0
    fetch_check_point = [100,200,300,400,500]
    with open('users.csv', mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
          username = row[0]
          if (log_enable):
            print(f"Fetching repositories for {username}...")
          repos = fetch_repositories(username)
          extracted_data = extract_repo_data(repos, username)
          all_repo_data.extend(extracted_data)
          total_repo_count = total_repo_count + all_repo_data.__len__()
          user_fetch_count = user_fetch_count + 1
          if(user_fetch_count in fetch_check_point):
              print("User Fetch completed till ",user_fetch_count)

    # Write the collected data to a CSV file
    with open('repositories.csv', mode='w', newline='', encoding='utf-8') as output_file:
        fieldnames = [
            "login", "full_name", "created_at",
            "stargazers_count", "watchers_count",
            "language", "has_projects",
            "has_wiki", "license_name"
        ]
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)

        writer.writeheader()  # Write the header
        for data in all_repo_data:
            writer.writerow(data)  # Write the data rows
    print("Total Rows Fetched ",total_repo_count)
    print("Data successfully written to repositories.csv.")

if __name__ == "__main__":
    main()


Error fetching repositories for login: 404
Total Rows Fetched  9
Data successfully written to repositories.csv.


In [1]:
import pandas as pd
repositories_df = pd.read_csv("repositories.csv")
repositories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66433 entries, 0 to 66432
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   login             66433 non-null  object
 1   full_name         66433 non-null  object
 2   created_at        66433 non-null  object
 3   stargazers_count  66433 non-null  int64 
 4   watchers_count    66433 non-null  int64 
 5   language          52460 non-null  object
 6   has_projects      66433 non-null  bool  
 7   has_wiki          66433 non-null  bool  
 8   license_name      36201 non-null  object
dtypes: bool(2), int64(2), object(5)
memory usage: 3.7+ MB


In [19]:
repositories_df.head(10)

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,dennybritz,dennybritz/ablog-content,2020-05-11T13:12:41Z,9,9,Jupyter Notebook,True,True,NaN
1,dennybritz,dennybritz/actioncrawler,2015-12-18T12:08:07Z,1,1,JavaScript,True,True,NaN
2,dennybritz,dennybritz/akka-cluster-deploy,2014-09-12T12:17:07Z,26,26,Scala,True,True,NaN
3,dennybritz,dennybritz/analysis-jobdata,2014-08-21T11:23:39Z,0,0,Python,True,True,NaN
4,dennybritz,dennybritz/arrow-datafusion,2023-02-25T11:11:16Z,0,0,Rust,True,False,Apache License 2.0
5,dennybritz,dennybritz/bella,2015-11-02T10:25:23Z,55,55,JavaScript,True,True,NaN
6,dennybritz,dennybritz/boilerpipe-api,2015-07-29T10:03:26Z,5,5,Scala,True,True,NaN
7,dennybritz,dennybritz/booknotes,2014-10-29T09:37:19Z,139,139,NaN,True,True,NaN
8,dennybritz,dennybritz/chatbot-retrieval,2016-04-19T00:52:38Z,1577,1577,Jupyter Notebook,True,True,MIT License
9,dennybritz,dennybritz/cnn-text-classification-tf,2015-11-24T15:43:25Z,5647,5647,Python,True,True,Apache License 2.0


In [20]:
repositories_df =repositories_df.fillna('')
repositories_df.head(5)

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,dennybritz,dennybritz/ablog-content,2020-05-11T13:12:41Z,9,9,Jupyter Notebook,True,True,
1,dennybritz,dennybritz/actioncrawler,2015-12-18T12:08:07Z,1,1,JavaScript,True,True,
2,dennybritz,dennybritz/akka-cluster-deploy,2014-09-12T12:17:07Z,26,26,Scala,True,True,
3,dennybritz,dennybritz/analysis-jobdata,2014-08-21T11:23:39Z,0,0,Python,True,True,
4,dennybritz,dennybritz/arrow-datafusion,2023-02-25T11:11:16Z,0,0,Rust,True,False,Apache License 2.0


In [ ]:
#3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.
licence =repositories_df['license_name'].value_counts().sort_values(ascending=False)
#licence = licence.sort_values(by =login,asc='False')
licence

,count
license_name,
,30232
MIT License,19818
Apache License 2.0,5405
Other,5333
"BSD 3-Clause ""New"" or ""Revised"" License",1385
GNU General Public License v3.0,1308
GNU General Public License v2.0,500
"BSD 2-Clause ""Simplified"" License",489
Creative Commons Zero v1.0 Universal,282


In [ ]:
language =repositories_df['language'].value_counts().sort_values(ascending=False)
#licence = licence.sort_values(by =login,asc='False')
language

,count
language,
,13973
JavaScript,8823
Ruby,5234
Python,4081
Go,3330
...,...
XC,1
GAP,1
SRecode Template,1


In [ ]:
#5. Which programming language is most popular among these users?
# repositories_df1= repositories_df
# repositories_df1['created_at'] = pd.to_datetime(repositories_df['created_at'])

# # Extract only the date
# repositories_df1['date_only'] = repositories_df1['created_at'].dt.date
# repositories_df1['date_only'] = pd.to_datetime(repositories_df1['date_only'])
# repositories_df1.head(5)
print(repositories_df.shape)
repositories_df1= repositories_df[repositories_df['created_at']>='2021-01-01']
print(repositories_df1.shape)

(66433, 10)
(15336, 10)


In [ ]:
#6.Which programming language is the second most popular among users who joined after 2020?


# Step 1: Filter users who joined after 2020
filtered_users = users_df[users_df['created_at'] >= '2020-01-01']

# Step 2: Merge the filtered users with the repositories DataFrame based on 'login'
merged_df = pd.merge(filtered_users[['login']], repositories_df, on='login', how='inner')

# Step 3: Count the occurrences of each language in the merged DataFrame
language_counts = merged_df['language'].value_counts()

# Step 4: Get the second most popular language
second_most_popular_language = language_counts.index[2] if len(language_counts) > 1 else None

print("The second most popular language among users who joined after 2020 is:", second_most_popular_language)


The second most popular language among users who joined after 2020 is: Rust


In [ ]:
#Which programming language is the second most popular among users who joined after 2020?
# Step 1: Filter for users who joined after 2020
filtered_df = repositories_df[repositories_df['created_at'] >= '2020-01-01']

# Step 2: Count occurrences of each programming language
language_counts = filtered_df['language'].value_counts()
print(language_counts)
# Step 3: Get the second most popular language
second_most_popular_language = language_counts.index[1] if len(language_counts) > 1 else None

print("The second most popular language among users who joined after 2020 is:", second_most_popular_language)

language
                     9169
TypeScript           2097
JavaScript           1236
Python               1073
Rust                  859
                     ... 
XML                     1
Visual Basic .NET       1
Scheme                  1
CUE                     1
Elm                     1
Name: count, Length: 141, dtype: int64
The second most popular language among users who joined after 2020 is: TypeScript


In [ ]:
rep_df = repositories_df['language'].str.upper().unique()
rep_df

array(['JUPYTER NOTEBOOK', 'JAVASCRIPT', 'SCALA', 'PYTHON', 'RUST', '',
       'SHELL', 'CSS', 'GO', 'HTML', 'SVELTE', 'HASKELL', 'RUBY', 'TEX',
       'C++', 'JAVA', 'KOTLIN', 'LUA', 'DART', 'LESS', 'TYPESCRIPT', 'C',
       'ASSEMBLY', 'EMACS LISP', 'DOCKERFILE', 'SCHEME', 'MAKEFILE',
       'NUNJUCKS', 'COFFEESCRIPT', 'C#', 'OBJECTIVE-C', 'PERL', 'PHP',
       'PROCESSING', 'MATLAB', 'CMAKE', 'QML', 'CYTHON', 'XC', 'OCAML',
       'VUE', 'WEBASSEMBLY', 'SWIFT', 'METAL', 'OBJECTIVE-C++', 'ZIG',
       'SCSS', 'MARKDOWN', 'YAML', 'CODEQL', 'EJS', 'HCL', 'ASTRO',
       'COMMON LISP', 'VIML', 'VIM SCRIPT', 'CUDA', 'EMBERSCRIPT',
       'SHADERLAB', 'GROOVY', 'D', 'ANTLR', 'STARLARK', 'CRYSTAL',
       'POWERSHELL', 'VIM SNIPPET', 'COBOL', 'ASP', 'CLOJURE', 'SQUIRREL',
       'ADA', 'VALA', 'ERLANG', 'PROLOG', 'R', 'SLASH', 'PUPPET',
       'SMARTY', 'GHERKIN', 'NGINX', 'ROFF', 'HLSL', 'GLSL', 'ELIXIR',
       'HAXE', 'MOVE', 'ACTIONSCRIPT', 'LLVM', 'PERL6', 'XSLT',
       'APPLESCRIPT'

In [ ]:
#7.Which language has the highest average number of stars per repository?
average_stars = repositories_df.groupby('language')['stargazers_count'].mean().sort_values(ascending=False)
average_stars

,stargazers_count
language,
Assembly,1732.970588
MDX,945.000000
Ragel in Ruby Host,342.000000
Lua,255.125874
Forth,144.500000
...,...
Twig,0.000000
ChucK,0.000000
V,0.000000


In [ ]:
#8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

users_df.info()
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])

# Sort users by leader_strength in descending order and select the top 5
top_leaders = users_df.sort_values(by='leader_strength', ascending=False).head(5)['login'].tolist()

top_leaders


<class 'pandas.core.frame.DataFrame'>
Index: 530 entries, 0 to 539
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   login         530 non-null    object             
 1   name          530 non-null    object             
 2   company       530 non-null    object             
 3   location      530 non-null    object             
 4   email         530 non-null    object             
 5   hireable      530 non-null    object             
 6   bio           530 non-null    object             
 7   public_repos  530 non-null    int64              
 8   followers     530 non-null    int64              
 9   following     530 non-null    int64              
 10  created_at    530 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(3), object(7)
memory usage: 49.7+ KB


['blueimp', 'dai-shi', 'asahilina', 'pilcrowonpaper', 'marcan']

In [ ]:
#9. What is the correlation between the number of followers and the number of public repositories among users in Tokyo?
correlation = users_df['followers'].corr(users_df['public_repos'])
correlation

0.05126542990505047

In [ ]:
#10. Does creating more repos help users get more followers?
#Using regression, estimate how many additional followers a user gets per additional public repository.

from sklearn.linear_model import LinearRegression
import pandas as pd

model = LinearRegression()

# Fit the model using 'public_repos' to predict 'followers'
X = users_df[['public_repos']]  # Independent variable
y = users_df['followers']       # Dependent variable

model.fit(X, y)

# Get the regression coefficient
followers_per_repo = model.coef_[0]
followers_per_repo

0.280974345599531

In [3]:
#11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

correlation = repositories_df['has_projects'].corr(repositories_df['has_wiki'])
print(correlation)
repositories_df_1=repositories_df[['has_projects','has_wiki']]
repositories_df_1['has_projects'] = repositories_df_1['has_projects'].astype(int)
repositories_df_1['has_wiki'] = repositories_df_1['has_wiki'].astype(int)
print(repositories_df_1.info())

# Step 2: Calculate the correlation between the two columns
correlation = repositories_df_1['has_projects'].corr(repositories_df_1['has_wiki'])
correlation


0.41600650646200427
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66433 entries, 0 to 66432
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   has_projects  66433 non-null  int64
 1   has_wiki      66433 non-null  int64
dtypes: int64(2)
memory usage: 1.0 MB
None


<ipython-input-3-5ff134906664>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repositories_df_1['has_projects'] = repositories_df_1['has_projects'].astype(int)
<ipython-input-3-5ff134906664>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repositories_df_1['has_wiki'] = repositories_df_1['has_wiki'].astype(int)


0.41600650646200427

In [ ]:
users_df['hireable'] = users_df['hireable'].apply(lambda x: True if x is True else False)


In [ ]:
users_df['hireable'] = users_df['hireable'].fillna(False)
hireable_counts = users_df['hireable'].value_counts()
hireable_counts

,count
hireable,
False,360
True,180


In [ ]:
users_df.shape

(540, 11)

In [47]:
import pandas as pd

def calculate_following_difference(csv_file):
    # Load the CSV data into a DataFrame
    df = pd.read_csv(csv_file)

    # Calculate average following for hireable users
    avg_following_hireable = df[df['hireable'] == True]['following'].mean()

    # Calculate average following for non-hireable users
    avg_following_non_hireable = df[df['hireable'] == False]['following'].mean()

    # Debugging output
    print(f"Average following for hireable users: {avg_following_hireable}")
    print(f"Average following for non-hireable users: {avg_following_non_hireable}")

    # Calculate the difference
    difference = avg_following_hireable - avg_following_non_hireable

    # Round the result to three decimal places
    return round(difference, 3)

# Example usage
csv_file_path = 'users.csv'  # Replace with the path to your CSV file
result = calculate_following_difference(csv_file_path)

print(f"Difference in average following: {result}")


Average following for hireable users: 182.13333333333333
Average following for non-hireable users: 261.27222222222224
Difference in average following: -79.139


In [22]:
#12. Do hireable users follow more people than those who are not hireable?
users_df = pd.read_csv("users.csv")

users_df = users_df.dropna(subset=['hireable', 'following'])

# Calculate average following for hireable users
avg_following_hireable = users_df[users_df['hireable'] == True]['following'].mean()

# Calculate average following for non-hireable users
avg_following_non_hireable = users_df[users_df['hireable'] == False]['following'].mean()

# Calculate the difference
following_difference = avg_following_hireable - avg_following_non_hireable
print(f"Difference in average following (hireable - non-hireable): {following_difference:.3f}")


Difference in average following (hireable - non-hireable): -79.139


In [23]:
##13. Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

#users_with_bios = users_df[users_df['bio'].fillna('').str.strip() != ""]
from scipy.stats import linregress
import numpy as np
users_df = pd.read_csv("users.csv")

# Step 1: Filter out rows with missing bios and calculate word count for each bio
users_df = users_df.dropna(subset=['bio'])
users_df['bio_length'] = users_df['bio'].apply(lambda x: len(x.split()))

# Step 2: Perform linear regression using linregress
slope, intercept, r_value, p_value, std_err = linregress(users_df['bio_length'], users_df['followers'])

# Step 3: Output the slope rounded to three decimal places
print(f"The regression slope of followers on bio word count is: {slope:.3f}")



The regression slope of followers on bio word count is: 18.645


In [20]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataframe (assuming it's loaded as users_df)
users_df = users_df.dropna(subset=['bio', 'followers'])

# Count words in each bio (split by whitespace and count the words)
users_df['bio_word_count'] = users_df['bio'].str.split().apply(len)

# Reshape bio_word_count to 2D array for the regression model
X = users_df['bio_word_count'].values.reshape(-1, 1)
y = users_df['followers'].values

# Fit the linear regression model
reg_model = LinearRegression().fit(X, y)

# Get the regression slope
slope = reg_model.coef_[0]
print(f"Regression slope of followers on bio word count: {slope:.3f}")

Regression slope of followers on bio word count: 19.193


In [ ]:
##14.Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

# Convert 'created_at' to datetime format
repositories_df['created_at'] = pd.to_datetime(repositories_df['created_at'], errors='coerce')

# Filter out rows with null 'created_at' values
repositories_df = repositories_df.dropna(subset=['created_at'])

# Determine if the 'created_at' date is a weekend (Saturday=5 or Sunday=6)
repositories_df['is_weekend'] = repositories_df['created_at'].dt.dayofweek >= 5
# Filter to include only repositories created on weekends
weekend_repos = repositories_df[users_df['is_weekend'] == True]

# Count the number of repositories created on weekends per user
# Group by 'login', count 'created_at' instances, then sort by count in descending order
top_weekend_creators = weekend_repos['login'].value_counts().head(5)

# Output the result as a comma-separated string
top_weekend_creators_str = ', '.join(top_weekend_creators.index)

print("Top 5 users with most weekend-created repositories:", top_weekend_creators_str)


Top 5 users with most weekend-created repositories: azu, suzuki-shunsuke, yuiseki, xuwei-k, kt3k


In [39]:
#15.Do people who are hireable share their email addresses more often?
#[fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

users_df = pd.read_csv("users.csv")
print(users_df[users_df['email'].notna()].shape[0])
print(users_df[users_df['hireable']].shape[0])
print( users_df[users_df['hireable'] & users_df['email'].notna()].shape[0])
hireable_users_with_email = users_df[users_df['hireable'] & users_df['email'].notna()].shape[0]
total_hireable_users = users_df[users_df['hireable']].shape[0]


non_hireable_users_with_email = users_df[~users_df['hireable'] & users_df['email'].notna()].shape[0]
total_non_hireable_users = users_df[~users_df['hireable']].shape[0]

    # Avoid division by zero
if total_hireable_users == 0 or total_non_hireable_users == 0:
  print("No users in one of the categories.")

    # Calculate fractions
fraction_hireable_with_email = hireable_users_with_email / total_hireable_users
fraction_non_hireable_with_email = non_hireable_users_with_email / total_non_hireable_users

    # Calculate the difference
difference = fraction_hireable_with_email - fraction_non_hireable_with_email

    # Round the result to three decimal places
print(round(difference, 3))


309
180
119
0.133


In [44]:
import pandas as pd

def calculate_email_fraction_groupby(csv_file):
    # Load the CSV data into a DataFrame
    df = pd.read_csv(csv_file)

    # Create a new column to identify users with email
    df['has_email'] = df['email'].notna()

    # Group by hireable status and calculate counts
    summary = df.groupby('hireable').agg(
        total_users=('login', 'count'),
        users_with_email=('has_email', 'sum')
    ).reset_index()

    # Calculate the fractions
    summary['email_fraction'] = summary['users_with_email'] / summary['total_users']

    # Debugging output
    print(summary)

    # Get the fractions for hireable and non-hireable
    hireable_fraction = summary[summary['hireable'] == True]['email_fraction'].values[0]
    non_hireable_fraction = summary[summary['hireable'] == False]['email_fraction'].values[0]

    # Calculate the difference
    difference = hireable_fraction - non_hireable_fraction

    # Round the result to three decimal places
    return round(difference, 3)

csv_file_path = 'users.csv'
result = calculate_email_fraction_groupby(csv_file_path)

print(f"Difference in fractions: {result}")


   hireable  total_users  users_with_email  email_fraction
0     False          360               190        0.527778
1      True          180               119        0.661111
Difference in fractions: 0.133


In [43]:
import pandas as pd

def calculate_email_difference_from_csv(csv_file):
    # Load the CSV data into a DataFrame
    df = pd.read_csv(csv_file)

    # Count hireable users with valid email addresses
    hireable_users_with_email = df[(df['hireable'] == True) & (df['email'].notna())].shape[0]
    total_hireable_users = df[df['hireable'] == True].shape[0]

    # Count non-hireable users with valid email addresses
    non_hireable_users_with_email = df[(df['hireable'] == False) & (df['email'].notna())].shape[0]
    total_non_hireable_users = df[df['hireable'] == False].shape[0]

    # Debugging statements to print counts
    print(f"Total hireable users: {total_hireable_users}")
    print(f"Total non-hireable users: {total_non_hireable_users}")
    print(f"Hireable users with email: {hireable_users_with_email}")
    print(f"Non-hireable users with email: {non_hireable_users_with_email}")

    # Avoid division by zero
    if total_hireable_users == 0 or total_non_hireable_users == 0:
        return "No users in one of the categories."

    # Calculate fractions
    fraction_hireable_with_email = hireable_users_with_email / (total_hireable_users)
    fraction_non_hireable_with_email = non_hireable_users_with_email / (total_non_hireable_users+total_hireable_users)

    # Calculate the difference
    difference = fraction_hireable_with_email - fraction_non_hireable_with_email

    # Round the result to three decimal places
    return round(difference, 3)

csv_file_path = 'users.csv'
result = calculate_email_difference_from_csv(csv_file_path)

print(f"Difference in fractions: {result}")



Total hireable users: 180
Total non-hireable users: 360
Hireable users with email: 119
Non-hireable users with email: 190
Difference in fractions: 0.309


In [35]:
users_df.head(5)

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,dennybritz,Denny Britz,NaN,"Tokyo, Japan",NaN,False,"High-school dropout. Ex Google Brain, Stanford...",61,9472,6,2010-09-16T18:32:26Z
1,wasabeef,Daichi Furiya,"CYBERAGENT, INC.","Tokyo, Japan",NaN,True,Google Developers Expert for Android,46,9369,60,2012-06-09T13:46:37Z
2,dai-shi,Daishi Kato,FREELANCER,Tokyo,NaN,True,"React library author, maintaining three state ...",128,6939,1,2010-11-21T12:26:31Z
3,rui314,Rui Ueyama,BLUE WHALE SYSTEMS,Tokyo,rui314@gmail.com,False,NaN,44,5936,8,2009-01-18T01:29:27Z
4,domenic,Domenic Denicola,GOOGLE,"Tokyo, Japan",d@domenic.me,False,NaN,216,5404,79,2011-02-14T15:26:22Z


In [ ]:
## 16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.)
##What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)


users_with_names = users_df.dropna(subset=['name'])

# Extract the last word in each name as the surname
users_with_names['surname'] = users_with_names['name'].str.strip().str.split().str[-1]

# Count occurrences of each surname
surname_counts = users_with_names['surname'].value_counts()

# Find the highest frequency
max_count = surname_counts.max()

# Get all surnames that have the maximum frequency
most_common_surnames = surname_counts[surname_counts == max_count].index.sort_values()

# Convert result to a comma-separated string
most_common_surnames_str = ', '.join(most_common_surnames)

print("Most common surname(s):", most_common_surnames_str)


Most common surname(s): Kato, Tanaka
